**CAMERA CALIBRATION**

**Take images**

In [ ]:
import numpy as np
import cv2
import glob

id_image=0

print('Press (s) to save images and (q) to quit!')

# Open cameras
CamR= cv2.VideoCapture(1)
CamL= cv2.VideoCapture(0)

if CamR.isOpened() and CamL.isOpened():
    print ('CAMERA ON')
else:
    print('CAMERA OFF')

images_left = images_right = []
images_left = glob.glob('calib_images/left*.jpg')
images_right = glob.glob('calib_images/right*.jpg')

while True:
    retR, frameR= CamR.read()
    retL, frameL= CamL.read()

    # If found, add object points, image points (after refining them)
    if (retR == True) & (retL == True):
        cv2.imshow('VideoR',frameR)
        cv2.imshow('VideoL',frameL)

        if cv2.waitKey(10) & 0xFF == ord('s'):
            if images_left==[] and images_right==[]:
                str_id_image= str(id_image)
                print('Images saved for right and left cameras')
                cv2.imwrite('calib_images/right'+str_id_image+'.jpg',frameR)
                cv2.imwrite('calib_images/left'+str_id_image+'.jpg',frameL)
                id_image=id_image+1
            else:
                print('Images not saved, files already exist')
            
    # End the Programme
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# When everything done, release the capture
CamR.release()
CamL.release()
cv2.destroyAllWindows()

**Setup**

In [ ]:
import numpy as np
import cv2
import glob

pt_col = 6
pt_lin = 5
n_matchs = 0

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((pt_lin*pt_col,3), np.float32)
objp[:,:2] = np.mgrid[0:pt_lin,0:pt_col].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpointsL = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
objpointsR = [] # 3d point in real world space
imgpointsR = [] # 2d points in image plane.

images_left = images_right = []
images_left = glob.glob('calib_images/left*.jpg')
images_right = glob.glob('calib_images/right*.jpg')

for i in range(0, len(images_left), 1):
    imgL = cv2.imread(images_left[i])
    imgR = cv2.imread(images_right[i])
    print(images_left[i], ' ', images_right[i])
    
    grayR= cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
    grayL= cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv2.findChessboardCorners(grayL, (pt_lin,pt_col),None)
    retR, cornersR = cv2.findChessboardCorners(grayR, (pt_lin,pt_col),None)
    
    # If found, add object points, image points (after refining them)
    if retL == True & retR == True:
        objpointsL.append(objp)
        objpointsR.append(objp)

        cornersL2 = cv2.cornerSubPix(grayL,cornersL,(11,11),(-1,-1),criteria)
        imgpointsL.append(cornersL2)
        cornersR2 = cv2.cornerSubPix(grayR,cornersR,(11,11),(-1,-1),criteria)
        imgpointsR.append(cornersR2)
        
        # Draw and display the corners
        imgL = cv2.drawChessboardCorners(imgL, (pt_lin,pt_col), cornersL2,retL)
        imgR = cv2.drawChessboardCorners(imgR, (pt_lin,pt_col), cornersR2,retR)
        
        print('Match')
        n_matchs += 1
    
    cv2.imshow('imgL',imgL)
    cv2.imshow('imgR',imgR)
    cv2.waitKey(500)

print('\nNUMBER OF MATCHES: ', n_matchs)
cv2.destroyAllWindows()

**Calibration**

In [ ]:
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpointsL, imgpointsL, grayL.shape[::-1],None,None)
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpointsR, imgpointsR, grayR.shape[::-1],None,None)

# Save calibration data
np.savez('data_calib_left', mtxL=mtxL, distL=distL, rvecsL=rvecsL, tvecsL=tvecsL, objpointsL=objpointsL, imgpointsL=imgpointsL)
np.savez('data_calib_right', mtxR=mtxR, distR=distR, rvecsR=rvecsR, tvecsR=tvecsR, objpointsR=objpointsR, imgpointsR=imgpointsR)

print('Data of calibration saved as "data_calib_left" and "data_calib_right"!')